# Semantic Style GAN を用いた表情モーフィング生成

## 1. 事前学習済みモデルをダウンロードする
学習済みモデルをpklファイルをダウンロードして`pretrained/` に配置する．`pretrained/`がない場合は自分で作成すること．<br>
**DL : https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt**

## 2. 画像のサイズを事前学習済みモデルに合わせる
CelebAMask-HQでは512x512サイズであるので，入力画像もそのサイズにリサイズしてBGR形式を作成する．<br>
`images/`がない場合は作成し入力画像を配置して以下のプログラムを動かす．元画像が上書きされるので注意が必要．

In [ ]:
!python resize512.py --indir images

## 3. 顔画像を中心に配置
少しでも顔が中心でないく正しく推論できないため，画像を中心に配置する．

In [ ]:
!python align_images.py --indir images

## 4. 出力フォルダを作成する

invertするときの出力先は`results/inversion`，モーフィングの出力先は`results/interpolation`とする．

In [ ]:
mkdir -p results/inversion results/interpolation

## 5. 画像をモデルにinvertする（学習済みモデルに則して潜在変数を探索してファインチューニングをする）
`images/`にある顔画像でをinvertして潜在変数(npy)を作成して，モデル(pt)をファインチューニングする．ファインチューニングはptiアルゴリズムを用いる．<br>
潜在変数は`results/inversion/latent/`，ファインチューニング済みモデルは`results/inversion/weights/`に出力される．

In [ ]:
!PYTHONPATH=.:$PYTHONPATH python visualize/invert_pti.py --ckpt pretrained/CelebAMask-HQ-512x512.pt --imgdir images --outdir results/inversion --size 512 --step 3000  --finetune_step 5000

### (5.2) 潜在変数を確認する
潜在変数が本当に指定した顔に近づいているか確認する．<br>
顔画像だけなら`results/inversion/recon/`，`results/inversion/recon_finetune/`に出力されるが，パーツごとの画像やスペクトルなどを調べるには以下を使う．

In [ ]:
!python components_display.py

## 6.A 表情間モーフィングを作成する（線形補間）
`/pretrained`にモデルを移してモーフィングを作成する．<br>
逆順のモーフィングは作成されないのが逆再生で対応する．顔パーツ区間に分けながら潜在変数で線形モーフィングする．<br>
線形モーフィングは $ I^M=\alpha I^S+(1.0-\alpha) I^T \quad(0 \leq \alpha \leq 1.0)$ ではあるが，$ I^S $，$ I^T $は単に画素値ではなく潜在変数であるこに留意する．

In [1]:
cp -f results/inversion/weights/pti.pt pretrained/

In [ ]:
!python morphing_line.py

## 6.B 任意方向モーフィングを作成する（単体補間法）
`/mat`にmatlabから表情弁別閾値楕円の中間ファイルを入れる．<br>
中間ファイルは
- thresholds.mat : 表情弁別閾値測定の画像のベクトルリスト
- (expression).mat : 基本表情expressionの計量
- points.mat : 全顔画像の心理物理空間の座標
- points_info : 基本表情の添字と心理物理空間の座標を結びつけるファイル

ベクトル（軸方向）$\boldsymbol{e}$と頂点$\boldsymbol{u}_1 \dots \boldsymbol{u}_6$からパラメータ$\boldsymbol{\gamma}$を得る．このパラメータは線形写像であるとき保存される．潜在空間は非線形であるが，リーマン多様体であるため局所的に保存される．
$$
\begin{align*}
	\boldsymbol{e} 
	                    & =\left(\begin{array}{ccc}
			                             \mid             &       & \mid              \\
			                             \boldsymbol{u}_1 & \dots & \boldsymbol{u}_6  \\
			                             \mid             &       & \mid
		                             \end{array}\right)\left(\begin{array}{l}
			                                                     c_1    \\
			                                                     \vdots \\
			                                                     c_6
		                                                     \end{array}\right)                                                         \\
	                    & =: V \boldsymbol{\gamma}                                                                                           \\
	V                   & :=\left(\begin{array}{ccc}
			                              \mid             &       & \mid             \\
			                              \boldsymbol{u}_1 & \dots & \boldsymbol{u}_6 \\
			                              \mid             &       & \mid
		                              \end{array}\right), \boldsymbol{\gamma}:=\left(\begin{array}{l}
			                                                                             c_1    \\
			                                                                             \vdots \\
			                                                                             c_6
		                                                                             \end{array}\right)                                    \\
	\boldsymbol{\gamma} & =V^{-1} \boldsymbol{e} 
\end{align*}
$$
morphing_dict.pyで用いる表情や主軸の数などのパラメータはコードを修正すること．

In [ ]:
cp -f results/inversion/weights/pti.pt pretrained/

In [ ]:
!python morphing_dict.py

## memo
- 3において2をおこなわないと色空間の影響で上手に配置できないときがある．
- 6.Bにおいて表情弁別閾値楕円の測定するmatlabファイルは公開していないので，それでも使いたい場合は自分の使いたいデータ用に変更すること．